In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Layer, BatchNormalization, GlobalAveragePooling2D 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image 
import cv2
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils  import to_categorical
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import random



# ensure consistency across runs
np.random.seed(1)
tf.random.set_seed(1)

In [5]:
data = pd.read_csv('data_csv.csv')
data.head()

,Unnamed: 0,image,label
0,287,data/NA_Fish_Dataset/Black Sea Sprat/00006.png,Black Sea Sprat
1,57,data/NA_Fish_Dataset/Red Mullet/00043.png,Red Mullet
2,152,data/NA_Fish_Dataset/Red Sea Bream/100_1481.png,Red Sea Bream
3,308,data/NA_Fish_Dataset/Horse Mackerel/00031.png,Horse Mackerel
4,425,data/NA_Fish_Dataset/Trout/00015.png,Trout


In [6]:
data = data.drop(data.columns[0],axis=1)
data.head()

,image,label
0,data/NA_Fish_Dataset/Black Sea Sprat/00006.png,Black Sea Sprat
1,data/NA_Fish_Dataset/Red Mullet/00043.png,Red Mullet
2,data/NA_Fish_Dataset/Red Sea Bream/100_1481.png,Red Sea Bream
3,data/NA_Fish_Dataset/Horse Mackerel/00031.png,Horse Mackerel
4,data/NA_Fish_Dataset/Trout/00015.png,Trout


In [7]:
binary_labels = pd.get_dummies(data["label"], prefix="label")
label_cols = binary_labels.columns
print(binary_labels)
data = data.join(binary_labels)
data

     label_Black Sea Sprat  label_Gilt Head Bream  label_Horse Mackerel  \
0                        1                      0                     0   
1                        0                      0                     0   
2                        0                      0                     0   
3                        0                      0                     1   
4                        0                      0                     0   
..                     ...                    ...                   ...   
422                      0                      0                     1   
423                      0                      0                     1   
424                      0                      0                     1   
425                      0                      0                     0   
426                      0                      0                     0   

     label_Red Mullet  label_Red Sea Bream  label_Sea Bass  label_Shrimp  \
0                   0  

,image,label,label_Black Sea Sprat,label_Gilt Head Bream,label_Horse Mackerel,label_Red Mullet,label_Red Sea Bream,label_Sea Bass,label_Shrimp,label_Striped Red Mullet,label_Trout
0,data/NA_Fish_Dataset/Black Sea Sprat/00006.png,Black Sea Sprat,1,0,0,0,0,0,0,0,0
1,data/NA_Fish_Dataset/Red Mullet/00043.png,Red Mullet,0,0,0,1,0,0,0,0,0
2,data/NA_Fish_Dataset/Red Sea Bream/100_1481.png,Red Sea Bream,0,0,0,0,1,0,0,0,0
3,data/NA_Fish_Dataset/Horse Mackerel/00031.png,Horse Mackerel,0,0,1,0,0,0,0,0,0
4,data/NA_Fish_Dataset/Trout/00015.png,Trout,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
422,data/NA_Fish_Dataset/Horse Mackerel/00008.png,Horse Mackerel,0,0,1,0,0,0,0,0,0
423,data/NA_Fish_Dataset/Horse Mackerel/00007.png,Horse Mackerel,0,0,1,0,0,0,0,0,0
424,data/NA_Fish_Dataset/Horse Mackerel/00014.png,Horse Mackerel,0,0,1,0,0,0,0,0,0
425,data/NA_Fish_Dataset/Striped Red Mullet/00022.png,Striped Red Mullet,0,0,0,0,0,0,0,1,0


In [16]:
# Split data
x_train, x_test = train_test_split(data, test_size=0.2, shuffle=True, random_state=1, stratify = data['label'])
x_train, x_valid = train_test_split(x_train, test_size=0.2, shuffle=True, random_state=1, stratify = x_train['label'])

x_train.shape, x_test.shape, x_valid.shape, data.shape

((272, 11), (86, 11), (69, 11), (427, 11))

In [22]:
x_valid.head()

,image,label,label_Black Sea Sprat,label_Gilt Head Bream,label_Horse Mackerel,label_Red Mullet,label_Red Sea Bream,label_Sea Bass,label_Shrimp,label_Striped Red Mullet,label_Trout
306,data/NA_Fish_Dataset/Red Mullet/00021.png,Red Mullet,0,0,0,1,0,0,0,0,0
106,data/NA_Fish_Dataset/Red Sea Bream/SAM_5537.png,Red Sea Bream,0,0,0,0,1,0,0,0,0
44,data/NA_Fish_Dataset/Red Sea Bream/SAM_5571.png,Red Sea Bream,0,0,0,0,1,0,0,0,0
350,data/NA_Fish_Dataset/Red Sea Bream/SAM_5551.png,Red Sea Bream,0,0,0,0,1,0,0,0,0
125,data/NA_Fish_Dataset/Horse Mackerel/00009.png,Horse Mackerel,0,0,1,0,0,0,0,0,0


In [17]:
# Training set generator
train_generator = ImageDataGenerator(rescale = 1./255)

# Training set generator with augmentation
train_generator_aug = ImageDataGenerator(
      rescale = 1./255,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')

# Test set generator
test_generator = ImageDataGenerator(rescale=1./255)

In [26]:
# Create datasets
train_images = train_generator.flow_from_dataframe(
    x_train,  x_col='image', y_col=label_cols,
    target_size=(250, 250), color_mode='rgb',
    class_mode='raw', batch_size=64,seed = 1
    )
val_images = train_generator.flow_from_dataframe(
    x_valid,  x_col='image', y_col=label_cols,
    target_size=(250, 250), color_mode='rgb',
    class_mode='raw', batch_size=64, seed = 1
    )
test_images = test_generator.flow_from_dataframe(
    x_test, x_col='image', y_col=label_cols,
    target_size=(250, 250), color_mode='rgb',seed = 1,
    class_mode="raw", batch_size=64
    )
train_images_aug = train_generator_aug.flow_from_dataframe(
    x_train,  x_col='image', y_col=label_cols,
    target_size=(250, 250), color_mode='rgb',
    class_mode='raw', batch_size=64,seed = 1
    )

Found 272 validated image filenames.
Found 69 validated image filenames.
Found 86 validated image filenames.
Found 272 validated image filenames.


In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(250, 250, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 248, 248, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 124, 124, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 53824)             0

In [ ]:
# Train non-augmented model


tf.keras.backend.clear_session()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

num_epochs = 25
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=["accuracy"])
model_history = model.fit(train_images, epochs=num_epochs,callbacks = [reduce_lr],validation_data = val_images)
model.save('model_1.h5')

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(250, 250, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['acc'])

model2.summary()

In [ ]:
# Train augmented model

tf.keras.backend.clear_session()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)

num_epochs = 25
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=["accuracy"])
model_history_aug = model.fit(train_images_aug, epochs=num_epochs,callbacks = [reduce_lr],validation_data = val_images)
model2.save('model_1.h5')

In [30]:
model = tf.keras.models.load_model('model_1.h5')
scoreSeg = model.evaluate(x=val_images,verbose =1)

2/2 [==============================] - 0s 142ms/step - loss: 0.5846 - acc: 0.7826


In [31]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs =range(num_epochs)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.legend()
plt.show()

NameError: name 'model_history' is not defined

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

plt.figure()
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'g', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy Value')
plt.legend()
plt.show()

In [34]:
test_predictions = model.predict(test_images)
test_predictions = np.argmax(test_predictions, axis=1)
test_predictions

array([3, 1, 7, 6, 0, 6, 5, 4, 0, 4, 0, 5, 3, 8, 2, 6, 1, 4, 7, 3, 7, 1,
       0, 5, 8, 3, 2, 2, 6, 7, 4, 1, 6, 5, 4, 3, 8, 5, 7, 6, 4, 1, 2, 8,
       0, 3, 0, 8, 4, 1, 3, 3, 4, 6, 1, 0, 0, 1, 4, 7, 7, 2, 8, 6, 8, 5,
       6, 3, 3, 0, 8, 5, 2, 5, 2, 3, 1, 6, 4, 4, 7, 4, 0, 4, 4, 1])

In [36]:
test_gt = np.argmax(test_images.labels, axis=1)
test_gt

array([1, 1, 1, 0, 2, 7, 2, 2, 0, 0, 7, 0, 4, 2, 6, 7, 6, 0, 8, 7, 5, 0,
       0, 6, 1, 2, 6, 7, 3, 3, 1, 8, 5, 7, 0, 5, 0, 4, 3, 4, 6, 6, 3, 7,
       5, 3, 8, 3, 3, 6, 2, 4, 2, 5, 1, 1, 4, 1, 5, 7, 2, 8, 3, 2, 4, 4,
       8, 1, 7, 5, 6, 3, 5, 2, 3, 5, 4, 7, 6, 4, 1, 6, 8, 5, 4, 0])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_gt, test_predictions))

In [40]:
# Adapted From: https://scikit-learn.org/0.18/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`.
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

#python -m pip install seaborn
import seaborn as sn

cnf_matrix = confusion_matrix(test_gt, test_predictions)
df_cm = pd.DataFrame(cnf_matrix,label_cols,label_cols)
plt.figure(figsize=(10,8))
sn.set(font_scale=1.0) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10},fmt='d',cmap=plt.cm.Blues) # font size


plt.show()

In [44]:
print(label_cols)
from sklearn import metrics
print(metrics.classification_report(test_gt, test_predictions))

Index(['label_Black Sea Sprat', 'label_Gilt Head Bream',
       'label_Horse Mackerel', 'label_Red Mullet', 'label_Red Sea Bream',
       'label_Sea Bass', 'label_Shrimp', 'label_Striped Red Mullet',
       'label_Trout'],
      dtype='object')
              precision    recall  f1-score   support

           0       0.20      0.20      0.20        10
           1       0.30      0.30      0.30        10
           2       0.00      0.00      0.00        10
           3       0.09      0.10      0.10        10
           4       0.14      0.20      0.17        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.12      0.10      0.11        10
           8       0.00      0.00      0.00         6

    accuracy                           0.10        86
   macro avg       0.10      0.10      0.10        86
weighted avg       0.10      0.10      0.10        86

